# puztool.logic_grid: Grid Logic helpers

In [1]:
from puztool import *
from puztool.logic_grid import *

These are tools for working with the sort of logic puzzles where you have a bunch of categories, a bunch of unique labels in each category, and a bunch of declarations about various subsets of these. For example:

1. Brita is not an Aries
2. Sam, the Green haired person, and the Virgo are three different people.
3. The person with Blue hair is a Scorpio
4. Brita has Red or Blue hair.
5. Neither Zork nor the Crabbus has Green hair.
6. Galal and Zork have the same number of letters in their signs.
7. Sam has Taupe hair or is an Aries, but not both.

The goal, then, is to determine which of Brita, Galal, and Sam has Blue, Red, or Green hair, and which is an Ares, Scorpio, or Virgo.

We represent a simple grid of values with the `Grid` type, which takes as input a data frame containing all possible values for each category, in no particular order:

In [2]:
data = pt('''
Brita  Blue   Aries
Galal  Green  Scorpio
Sam    Red    Virgo
Zork   Taupe  Crabbus
''', names=['Name', 'Color', 'Sign'])
data

,Name,Color,Sign
0,Brita,Blue,Aries
1,Galal,Green,Scorpio
2,Sam,Red,Virgo
3,Zork,Taupe,Crabbus


In [3]:
g = Grid(data)

This `Grid` object has internal grids of booleans indicating which pairs of values are known to be true or false; initially it's all `None`.

In [4]:
g.grids['Name']['Color']

array([[None, None, None, None],
       [None, None, None, None],
       [None, None, None, None],
       [None, None, None, None]], dtype=object)

To set these values, we can use the helper methods `exclude`, `include`, and `requireOne`.

`exclude(*values)` takes any number of values and encodes that they are all mutually exclusive.
`include(*values)` indicates that the specified values all correspond to each other.
`requireOne(value, [options])` indicates that the first value corresponds to exactly one of the chosen options.

In all cases, you don't need to indicate which category your values are in - `Grid` will infer this from the value itself. Thus, we can indicate the most of the rules listed above with:

In [5]:
g.exclude("Brita", "Aries") # Brita and Ares cannot be the same row
g.exclude("Sam", "Green", "Virgo") # No two of these three can be in the same row
g.include("Scorpio", "Blue") # Scorpio and Blue must be in the same row
g.requireOne("Brita", ["Red", "Blue"]) # The row containing Brita must contain Red or Blue
g.exclude("Zork", "Crabbus", "Green") # None of these are in the same row
# Virgo and Aries are the only two signs with the same length, so Galal and Zork must have these
g.requireOne("Galal", ['Virgo', 'Aries'])
g.requireOne("Zork", ['Virgo', 'Aries'])

In an ideal world I'd build a tool for viewing this right here in the notebook, but this isn't that world so instead you can use `g.get_link()` to get a link to the grid-solving tool at http://www.jsingler.de/apps/logikloeser/, prefilled with your grid:

In [6]:
print(g.get_link())

http://www.jsingler.de/apps/logikloeser/?language=en#(at:s,ms:s,nc:3,ni:4,v:0,items:!(!(Brita,Galal,Sam,Zork),!(Blue,Green,Red,Taupe),!(Aries,Scorpio,Virgo,Crabbus)),n:!(a0b1,a0b3,a2b1,a3b1,a0c0,a1c1,a1c3,a2c2,a3c1,a3c3,b1c2,b1c3),p:!(b0c1))


Following the above link, you can see that the tool there has inferred a bunch of values for you, and you can probably solve the rest yourself using the last two rules. But in a more complicated case, you might want to use an automatic solver, so we'll demo that here, too.

The last two rules above can't be represented just by checking off boxes independently of the other rules. Fortunately, we can add them at the `Solver` step.

A `Solver` is a wrapper around a `Grid` that has a bunch of [Numberjack](http://numberjack.ucc.ie/doc/) variables for the different values and can impose constraints on them.

In [7]:
s = Solver(g)

In [8]:
print(s.vgrids['Name']['Color'])

[[Name_Color.0.0 in {0,1}, Name_Color.0.1 in {0,1}, Name_Color.0.2 in {0,1}, Name_Color.0.3 in {0,1}],
 [Name_Color.1.0 in {0,1}, Name_Color.1.1 in {0,1}, Name_Color.1.2 in {0,1}, Name_Color.1.3 in {0,1}],
 [Name_Color.2.0 in {0,1}, Name_Color.2.1 in {0,1}, Name_Color.2.2 in {0,1}, Name_Color.2.3 in {0,1}],
 [Name_Color.3.0 in {0,1}, Name_Color.3.1 in {0,1}, Name_Color.3.2 in {0,1}, Name_Color.3.3 in {0,1}]]


When we ask the solver to solve, it'll automatically generate constraints for all of the booleans in the grid, as well as the various sanity constraints (e.g., each name has exactly one color, if a name and a color match then their signs should also match, etc.). But we can also add new constraints on the Numberjack variables, which are accessed by `s.vars`:

In [9]:
s.add(s.vars['Sam','Taupe'] != s.vars['Sam','Aries']) # Sam is Taupe-haired or an Aries, but not both

Having added this, we can now solve `s` using Numberjack; once we've done so, `s.soln_grid` will be a `Grid` with values set according to the solution, and `s.soln` will be a dataframe of the actual solution:

In [10]:
s.solve()

In [11]:
print(s.soln_grid.get_link())

http://www.jsingler.de/apps/logikloeser/?language=en#(at:s,ms:s,nc:3,ni:4,v:0,items:!(!(Brita,Galal,Sam,Zork),!(Blue,Green,Red,Taupe),!(Aries,Scorpio,Virgo,Crabbus)),n:!(a0b1,a0b2,a0b3,a1b0,a1b2,a1b3,a2b0,a2b1,a2b2,a3b0,a3b1,a3b3,a0c0,a0c2,a0c3,a1c1,a1c2,a1c3,a2c0,a2c1,a2c2,a3c0,a3c1,a3c3,b0c0,b0c2,b0c3,b1c1,b1c2,b1c3,b2c0,b2c1,b2c3,b3c0,b3c1,b3c2),p:!(a0b0,a1b1,a2b3,a3b2,a0c1,a1c0,a2c3,a3c2,b0c1,b1c0,b2c2,b3c3))


In [12]:
s.soln

,Name,Color,Sign
0,Brita,Blue,Scorpio
1,Galal,Green,Aries
2,Sam,Taupe,Crabbus
3,Zork,Red,Virgo


## Future Work

Ideally I'd like to expand this to handle grids where the uniqueness constraint is relaxed, and grids where rows have numeric values instead of just choices from a set. For example, imagine if the above puzzle also included that each person had 0, 1, or 2 apples, and specified rules like "Sam had more apples than the Virgo" - you'd want to add another entry to the grid where each category can have a variable set to 0, 1, or 2, but *without* the constraint that each value appears exactly once, and you'd want to be able to specify constraints using syntax something like `s.vars['Sam','Apples'] > s.vars['Taupe','Apples']`. I don't think this is *hard*, per se, but it is *complicated*, and it's unlikely I'll ever get around to it.

It'd also be cool to warn you when there are multiple solutions. Maybe someday.